Imports necessários

In [1]:
import requests
from io import BytesIO
import zipfile
import pandas as pd

Download e extração

In [ ]:
'''# URL do arquivo compactado
url_do_arquivo_zip = 'https://portaldatransparencia.gov.br/download-de-dados/viagens/2023'

# Baixando o arquivo compactado
resposta = requests.get(url_do_arquivo_zip)

# Verificando se a solicitação foi bem-sucedida
if resposta.status_code == 200:
    # Lendo o conteúdo do arquivo compactado
    conteudo_zip = BytesIO(resposta.content)

    # Extraindo o conteúdo do arquivo compactado
    with zipfile.ZipFile(conteudo_zip, 'r') as zip_ref:
        # Local da extracao
        zip_ref.extractall('/csv_files')
    print("Conteúdo extraído com sucesso!")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {resposta.status_code}")'''

Leitura e transformacao dos arquivos .csv em DataFrame

In [2]:
arquivo_viagem = '../csv_files/2023_Viagem.csv'
arquivo_passagem = '../csv_files/2023_Passagem.csv'
arquivo_trecho = '../csv_files/2023_Trecho.csv'
arquivo_pagamento = '../csv_files/2023_Pagamento.csv'

# Carregando os dados do arquivo CSV em um DataFrame do pandas
df_viagem = pd.read_csv(arquivo_viagem, sep=';', encoding="latin-1")

df_passagem = pd.read_csv(arquivo_passagem, sep=';', encoding="latin-1")

df_trecho = pd.read_csv(arquivo_trecho, sep=';', encoding="latin-1")

df_pagamento = pd.read_csv(arquivo_pagamento, sep=';', encoding="latin-1")


Tratamento dos dados do dataframe Viagem

In [3]:
# Removendo colunas que não serão utilizadas
colunas_to_remove = ['Código do órgão superior', 'Nome do órgão superior', 'Código órgão solicitante', 'Nome órgão solicitante', 'Função', 'Descrição Função', 'Período - Data de início', 'Período - Data de fim', 'Destinos', 'Valor diárias', 'Valor passagens']
df_viagem.drop(colunas_to_remove, axis=1, inplace=True)

# Tratamento para as colunas com valores monetarios
colunas_to_numeric = ['Valor devolução', 'Valor outros gastos']
for coluna in colunas_to_numeric:
    df_viagem[coluna] = df_viagem[coluna].str.replace(',', '.')
    df_viagem[coluna] = df_viagem[coluna].apply(pd.to_numeric, errors='coerce')


Tratamento dos dados do dataframe Passagens

In [4]:
# Tratamento para as colunas com valores monetarios
colunas_to_numeric = ["Valor da passagem","Taxa de serviço"]
for coluna in colunas_to_numeric:
    df_passagem[coluna] = df_passagem[coluna].str.replace(',', '.')
    df_passagem[coluna] = df_passagem[coluna].apply(pd.to_numeric, errors='coerce')

# Removendo colunas desnecessárias
colunas_to_remove = ['Número da Proposta (PCDP)']
df_passagem.drop(colunas_to_remove, axis=1, inplace=True)

# Formatando data e hora para formato correto
df_passagem['Data da emissão/compra'] = pd.to_datetime(df_passagem['Data da emissão/compra'], format='%d/%m/%Y', errors='coerce')
df_passagem['Hora da emissão/compra'] = pd.to_datetime(df_passagem['Hora da emissão/compra'], format='%H:%M', errors='coerce').dt.time
 

Tratamento de dados do dataframe Trecho

In [5]:
# Removendo colunas desnecessárias
colunas_to_remove = ['Número da Proposta (PCDP)']
df_trecho.drop(colunas_to_remove, axis=1, inplace=True)

# Tratamento dos valores numericos
colunas_to_numeric = ['Número Diárias']
df_trecho[colunas_to_numeric] = df_trecho[colunas_to_numeric].apply(lambda x: x.str.replace(',', '.'))
df_trecho[colunas_to_numeric] = df_trecho[colunas_to_numeric].apply(pd.to_numeric, errors='coerce')

# Tratamento das datas
colunas_to_date = ['Origem - Data', 'Destino - Data']
df_trecho[colunas_to_date] = df_trecho[colunas_to_date].apply(pd.to_datetime, format='%d/%m/%Y', errors='coerce')

# Conversao para boleano
df_trecho['Missao?'] = df_trecho['Missao?'].apply(lambda x: True if x == 'Sim' else False)

Tratamento de dados do dataframe Pagamento

In [6]:
# Removendo colunas desnecessárias
colunas_to_remove = ['Número da Proposta (PCDP)']
df_pagamento.drop(colunas_to_remove, axis=1, inplace=True)

# Tratamento dos valores numericos
colunas_to_numeric = ['Valor']
df_pagamento[colunas_to_numeric] = df_pagamento[colunas_to_numeric].apply(lambda x: x.str.replace(',', '.'))
df_pagamento[colunas_to_numeric] = df_pagamento[colunas_to_numeric].apply(pd.to_numeric, errors='coerce')

Inserção no banco

In [16]:
from sqlalchemy import create_engine
import urllib.parse as parse

engine = create_engine('mssql+pyodbc://sa:%s@localhost/db_viagens_a_servico?driver=ODBC+Driver+17+for+SQL+Server' % parse.quote_plus('Vitor123'))

engine.connect()

df_viagem.to_sql('TB_VIAGE,', con=engine, if_exists='append', index=False)
df_passagem.to_sql('TB_PASSAGEM', con=engine, if_exists='append', index=False)
df_trecho.to_sql('TB_TRECHO', con=engine, if_exists='append', index=False)
df_pagamento.to_sql('TB_PAGAMENTO', con=engine, if_exists='append', index=False)



TypeError: __init__() got multiple values for argument 'schema'